In [1]:
cd ..

e:\GitHub\computervision


In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader

from modules.models_util import EfficientNet, ViT
from modules.training_util import makeDataloader

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights_path = [
    'notebooks\efficientnet\models/2024-02-01 10-31-27/best.pth',   # Fully trained 4k, 53.10%, 52.82% without transforms
    'notebooks\efficientnet\models/2024-02-01 09-55-37/best.pth',   # Fully trained 2k, 52.74%, 58.21%
    'notebooks\efficientnet\models/2024-02-02 16-34-31/best.pth',   # Fully trained even, 50.91%, 45.99%
    'notebooks\efficientnet\models/2024-31-01 18-53-33/best.pth',   # 4k, 55.84%, 53.83%
    'notebooks\efficientnet\models/2024-31-01 19-19-52/best.pth',   # 2k, 49.82%, 52.55%
    'notebooks/vit\models/01-31-2024 15-18-02/best.pth'             # ViT, 61.86%
]
# model = EfficientNet.load(weights_path[4])
model = ViT.load(weights_path[5])
model.to(device)
model.eval()

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [4]:
dataset_dir = 'E:\Datasets\cor-splits/temp-test'
dataloader = makeDataloader(dataset_dir, EfficientNet.data_transforms)
dataloader = makeDataloader(dataset_dir, ViT.data_transforms)

In [5]:
correct_predictions = 0
total_samples = 0

with torch.no_grad(): # no backtracking
    for inputs, labels in dataloader['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

accuracy = correct_predictions/total_samples
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 61.86%
